In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
#reading the dataset 
insurance_df=pd.read_csv('Seguro-Trimmed.csv')
insurance_df

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,...,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1,7,0,0,0,0,1,...,3,1,1,9,0,1,1,0,1,0
2,13,0,5,4,9,1,0,0,0,1,...,4,2,7,7,0,1,1,0,1,0
3,16,0,0,1,2,0,0,1,0,0,...,2,2,4,9,0,0,0,0,0,0
4,17,0,0,2,0,1,0,1,0,0,...,3,1,1,3,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54447,136609,0,1,1,6,0,0,1,0,0,...,7,5,4,4,0,1,1,0,0,0
54448,136611,0,0,2,2,0,0,0,0,0,...,4,0,2,11,0,1,1,0,0,0
54449,136614,0,2,1,5,1,4,0,0,1,...,3,1,4,11,1,1,0,0,1,1
54450,136615,0,1,1,3,1,0,0,1,0,...,9,3,3,4,0,1,0,1,0,0


In [3]:
y=insurance_df['target']
y
insurance_df.drop(columns='target',inplace=True)

In [4]:
insurance_df.replace(to_replace=-1,value=np.nan,inplace=True)

In [5]:
type(insurance_df)

pandas.core.frame.DataFrame

In [6]:
na_df=pd.DataFrame({'col_name':insurance_df.columns,
                   'na_count':insurance_df.isnull().sum(),
                   'na_percentage': insurance_df.isnull().sum()/insurance_df.shape[0]*100}) 
na_df.sort_values(by='na_percentage',ascending=False)
cols_to_drop=list(na_df[na_df['na_percentage']>50]['col_name'])
cols_to_drop.append('id') 


insurance_df.drop(columns=cols_to_drop,inplace=True)



                

In [7]:
#categorical columns
cat_cols=[col for col in insurance_df.columns if 'cat' in col] 
cat_cols

['ps_ind_02_cat',
 'ps_ind_04_cat',
 'ps_ind_05_cat',
 'ps_car_01_cat',
 'ps_car_02_cat',
 'ps_car_04_cat',
 'ps_car_05_cat',
 'ps_car_06_cat',
 'ps_car_07_cat',
 'ps_car_08_cat',
 'ps_car_09_cat',
 'ps_car_10_cat',
 'ps_car_11_cat']

In [8]:
#binary columns
bin_cols=[col for col in insurance_df.columns if 'bin' in col]
bin_cols

['ps_ind_06_bin',
 'ps_ind_07_bin',
 'ps_ind_08_bin',
 'ps_ind_09_bin',
 'ps_ind_10_bin',
 'ps_ind_11_bin',
 'ps_ind_12_bin',
 'ps_ind_13_bin',
 'ps_ind_16_bin',
 'ps_ind_17_bin',
 'ps_ind_18_bin',
 'ps_calc_15_bin',
 'ps_calc_16_bin',
 'ps_calc_17_bin',
 'ps_calc_18_bin',
 'ps_calc_19_bin',
 'ps_calc_20_bin']

In [9]:
#continous columns list
combined_cols=(cat_cols+bin_cols)
combined_cols
con_cols=[col for col in insurance_df.columns if col not in combined_cols]
con_cols

['ps_ind_01',
 'ps_ind_03',
 'ps_ind_14',
 'ps_ind_15',
 'ps_reg_01',
 'ps_reg_02',
 'ps_reg_03',
 'ps_car_11',
 'ps_car_12',
 'ps_car_13',
 'ps_car_14',
 'ps_car_15',
 'ps_calc_01',
 'ps_calc_02',
 'ps_calc_03',
 'ps_calc_04',
 'ps_calc_05',
 'ps_calc_06',
 'ps_calc_07',
 'ps_calc_08',
 'ps_calc_09',
 'ps_calc_10',
 'ps_calc_11',
 'ps_calc_12',
 'ps_calc_13',
 'ps_calc_14']

In [10]:
#train test split
X_train,X_test,y_train,y_test=train_test_split(insurance_df,y,test_size=0.2,random_state=43)

In [11]:
#finding missing values count
insurance_df.isnull().sum()

ps_ind_01             0
ps_ind_02_cat        17
ps_ind_03             0
ps_ind_04_cat         8
ps_ind_05_cat       498
ps_ind_06_bin         0
ps_ind_07_bin         0
ps_ind_08_bin         0
ps_ind_09_bin         0
ps_ind_10_bin         0
ps_ind_11_bin         0
ps_ind_12_bin         0
ps_ind_13_bin         0
ps_ind_14             0
ps_ind_15             0
ps_ind_16_bin         0
ps_ind_17_bin         0
ps_ind_18_bin         0
ps_reg_01             0
ps_reg_02             0
ps_reg_03          9959
ps_car_01_cat         8
ps_car_02_cat         0
ps_car_04_cat         0
ps_car_05_cat     24220
ps_car_06_cat         0
ps_car_07_cat       994
ps_car_08_cat         0
ps_car_09_cat        53
ps_car_10_cat         0
ps_car_11_cat         0
ps_car_11             0
ps_car_12             0
ps_car_13             0
ps_car_14          3923
ps_car_15             0
ps_calc_01            0
ps_calc_02            0
ps_calc_03            0
ps_calc_04            0
ps_calc_05            0
ps_calc_06      

In [12]:
#filling missing values of continuous columns using mean
for col in con_cols:
    X_train[col]=X_train[col].fillna(X_train[col].mean())
    X_test[col]=X_test[col].fillna(X_train[col].mean())
X_test[col]
    


<ipython-input-12-fea28d03b911>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[col]=X_train[col].fillna(X_train[col].mean())
<ipython-input-12-fea28d03b911>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[col]=X_test[col].fillna(X_train[col].mean())


51552    13
6235     10
31913    10
17733     6
6278      4
         ..
27623     7
45804     7
36739    14
26308    10
48453     5
Name: ps_calc_14, Length: 10891, dtype: int64

In [13]:
for col in combined_cols:
    X_train[col]=X_train[col].fillna(X_train[col].mode()[0])
    X_test[col]=X_test[col].fillna(X_train[col].mode()[0])

<ipython-input-13-67304b0ce2b1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[col]=X_train[col].fillna(X_train[col].mode()[0])
<ipython-input-13-67304b0ce2b1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[col]=X_test[col].fillna(X_train[col].mode()[0])


In [14]:
sm=SMOTE(sampling_strategy=0.2,random_state=40)
X_train_smote,y_train_smote=sm.fit_resample(X_train,y_train)
y_train_smote.value_counts(normalize=True)

0    0.833333
1    0.166667
Name: target, dtype: float64

In [15]:
#one-hot encoding
one_hot_train=pd.get_dummies(X_train_smote[cat_cols])
one_hot_test=pd.get_dummies(X_test[cat_cols])
one_hot_train.shape
oe_train,oe_test=one_hot_train.align(one_hot_test,join='inner',axis=1,fill_value=0)  



In [16]:
#Min Max Scaling
scaler=MinMaxScaler()
for col in con_cols:
    X_train_smote[col]=scaler.fit_transform(np.array(X_train_smote[col]).reshape(-1,1))
    X_test[col]=scaler.transform(np.array(X_test[col]).reshape(-1,1))



<ipython-input-16-269f92ceff1d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[col]=scaler.transform(np.array(X_test[col]).reshape(-1,1))
<ipython-input-16-269f92ceff1d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[col]=scaler.transform(np.array(X_test[col]).reshape(-1,1))
<ipython-input-16-269f92ceff1d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

<ipython-input-16-269f92ceff1d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[col]=scaler.transform(np.array(X_test[col]).reshape(-1,1))
<ipython-input-16-269f92ceff1d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[col]=scaler.transform(np.array(X_test[col]).reshape(-1,1))


In [17]:
train_final=pd.concat([X_train_smote[con_cols],oe_train],axis=1)
test_final=pd.concat([X_test[con_cols],oe_test],axis=1)

In [18]:
logreg=LogisticRegression()
logreg.fit(train_final,y_train_smote)

C:\Users\SRIRAM\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [19]:
pred=logreg.predict(test_final)

confusion_matrix(y_test,pred)

array([[10361,   128],
       [  397,     5]], dtype=int64)

In [20]:
rf=RandomForestClassifier()
param_grid_rf=({'n_estimators':[50,100,150,200],
               'max_depth':[3,4,5,6],
               'min_samples_leaf':[5,7,8,9,11],
               'max_features':["auto","sqrt","log2"]})
gdcv_rf=GridSearchCV(rf,param_grid=param_grid_rf,cv=4,verbose=5)
gdcv_rf.fit(train_final,y_train_smote)

Fitting 4 folds for each of 240 candidates, totalling 960 fits
[CV 1/4] END max_depth=3, max_features=auto, min_samples_leaf=5, n_estimators=50; total time=   2.1s
[CV 2/4] END max_depth=3, max_features=auto, min_samples_leaf=5, n_estimators=50; total time=   1.6s
[CV 3/4] END max_depth=3, max_features=auto, min_samples_leaf=5, n_estimators=50; total time=   1.8s
[CV 4/4] END max_depth=3, max_features=auto, min_samples_leaf=5, n_estimators=50; total time=   1.9s
[CV 1/4] END max_depth=3, max_features=auto, min_samples_leaf=5, n_estimators=100; total time=   3.8s
[CV 2/4] END max_depth=3, max_features=auto, min_samples_leaf=5, n_estimators=100; total time=   3.9s
[CV 3/4] END max_depth=3, max_features=auto, min_samples_leaf=5, n_estimators=100; total time=   3.8s
[CV 4/4] END max_depth=3, max_features=auto, min_samples_leaf=5, n_estimators=100; total time=   3.8s
[CV 1/4] END max_depth=3, max_features=auto, min_samples_leaf=5, n_estimators=150; total time=   5.6s
[CV 2/4] END max_depth=

[CV 1/4] END max_depth=3, max_features=sqrt, min_samples_leaf=5, n_estimators=50; total time=   1.8s
[CV 2/4] END max_depth=3, max_features=sqrt, min_samples_leaf=5, n_estimators=50; total time=   1.7s
[CV 3/4] END max_depth=3, max_features=sqrt, min_samples_leaf=5, n_estimators=50; total time=   1.6s
[CV 4/4] END max_depth=3, max_features=sqrt, min_samples_leaf=5, n_estimators=50; total time=   1.7s
[CV 1/4] END max_depth=3, max_features=sqrt, min_samples_leaf=5, n_estimators=100; total time=   4.1s
[CV 2/4] END max_depth=3, max_features=sqrt, min_samples_leaf=5, n_estimators=100; total time=   3.5s
[CV 3/4] END max_depth=3, max_features=sqrt, min_samples_leaf=5, n_estimators=100; total time=   3.4s
[CV 4/4] END max_depth=3, max_features=sqrt, min_samples_leaf=5, n_estimators=100; total time=   3.5s
[CV 1/4] END max_depth=3, max_features=sqrt, min_samples_leaf=5, n_estimators=150; total time=   5.3s
[CV 2/4] END max_depth=3, max_features=sqrt, min_samples_leaf=5, n_estimators=150; tot

KeyboardInterrupt: 

In [21]:
import imblearn

In [22]:
!pip install imblearn

  Using cached imblearn-0.0-py2.py3-none-any.whl (1.9 kB)


In [28]:
grdnt_bst=GradientBoostingClassifier()
grdnt_bst.fit(train_final,y_train_smote)

GradientBoostingClassifier()

In [31]:
pred_grdnt_bst=grdnt_bst.predict(test_final)
confusion_matrix(y_test,pred_grdnt_bst)

array([[10489,     0],
       [  402,     0]], dtype=int64)